In [1]:
import os
import sys
import json

# run locally without install asynfed package
root = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
# root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))
sys.path.append(root)


# tensorflow 
from asynfed.client.frameworks.tensorflow.tensorflow_framework import TensorflowFramework
from experiment.ember_dataset.ember_model import EmberModel
from data_preprocessing import *

2023-08-04 10:05:50.506006: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-04 10:05:50.540616: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 10:05:51.048014: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow as tf

In [3]:

with open('conf.json', 'r') as json_file:
    config = json.load(json_file)


In [4]:

epoch = 200
batch_size = 128

learning_rate_config = config.get('training_params').get('learning_rate_config', {}) 
if learning_rate_config == {}:
    learning_rate_config['fix_lr'] = False
    learning_rate_config['lr'] = 0.1
    learning_rate_config['decay_steps'] = 600000 * epoch / batch_size

In [5]:

# Define model
model = EmberModel(input_features= None, output_features= None, lr_config= learning_rate_config)


Config in the resnet model
{'fix_lr': False, 'lr': 0.1, 'decay_steps': 937500.0}
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 2381)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 2381, 8)      2056        ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 4, 128)       512128      ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 4, 128)       512128      ['embedding[0][0]']              
             

2023-08-04 10:05:52.768285: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22301 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:19:00.0, compute capability: 8.6
2023-08-04 10:05:52.768804: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 22301 MB memory:  -> device: 1, name: NVIDIA RTX A5000, pci bus id: 0000:8d:00.0, compute capability: 8.6


In [6]:
X = []
y = []

data_folder = "/home/student02/thaile/working_with_ember_dataset/data"
for i in range(10):
    X_chunk, y_chunk = load_data(f'{data_folder}/chunk_{i}.pickle')
    X.append(X_chunk)
    y.append(y_chunk)

X_val, y_val = load_data(f'{data_folder}/test_set.pickle')

X = np.concatenate(X, axis=0)
y = np.concatenate(y, axis=0)
print('X shape:', X.shape, '-- y shape:', y.shape)
print('X val shape:', X_val.shape, '-- y val shape:', y_val.shape)



X shape: (600000, 2381) -- y shape: (600000,)
X val shape: (200000, 2381) -- y val shape: (200000,)


In [7]:
train_ds = create_dataset(X, y)
test_ds = create_dataset(X_val, y_val)

In [8]:
data_size = len(X)
data_size

600000

In [9]:
# Define framework
tensorflow_framework = TensorflowFramework(model=model, 
                                           data_size= data_size, 
                                           train_ds= train_ds, 
                                           test_ds= test_ds, 
                                           config=config)


In [10]:
epoch

200

In [11]:
for epoch in range(epoch):
    print(f"Enter epoch {epoch + 1}, learning rate = {tensorflow_framework.get_learning_rate()}")
    tensorflow_framework.model.train_loss.reset_states()
    tensorflow_framework.model.train_performance.reset_states()
    tensorflow_framework.model.test_loss.reset_states()
    tensorflow_framework.model.test_performance.reset_states()

    for images, labels in tensorflow_framework.train_ds:
        train_acc, train_loss= tensorflow_framework.fit(images, labels)

    for test_images, test_labels in tensorflow_framework.test_ds:
        test_acc, test_loss = tensorflow_framework.evaluate(test_images, test_labels)

    print("*" * 20)
    print("Epoch {} - Train Acc: {:.2f} -- Train Loss {} Test Acc {:.2f}  Test Loss {}".format(epoch+1,
                                                                                       train_acc * 100,
                                                                                       train_loss,
                                                                                       test_acc * 100,
                                                                                       test_loss))
    print("*" * 20)


Enter epoch epoch, learning rate = 0.10000000149011612


2023-08-04 10:06:05.116768: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [600000]
	 [[{{node Placeholder/_1}}]]
2023-08-04 10:06:05.117018: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [600000]
	 [[{{node Placeholder/_1}}]]
2023-08-04 10:06:09.105559: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-08-04 10:06:09.721679: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-0

********************
Epoch 1 - Train Acc: 50.01 -- Train Loss 311.4507141113281 Test Acc 50.00  Test Loss 310.41607666015625
********************
Enter epoch epoch, learning rate = 0.09999383985996246
********************
Epoch 2 - Train Acc: 50.00 -- Train Loss 317.0620422363281 Test Acc 50.00  Test Loss 310.416259765625
********************
Enter epoch epoch, learning rate = 0.09997532516717911
********************
Epoch 3 - Train Acc: 50.08 -- Train Loss nan Test Acc 50.00  Test Loss nan
********************
Enter epoch epoch, learning rate = 0.09994449466466904
********************
Epoch 4 - Train Acc: 50.00 -- Train Loss nan Test Acc 50.00  Test Loss nan
********************
Enter epoch epoch, learning rate = 0.09990132600069046
********************
Epoch 5 - Train Acc: 50.00 -- Train Loss nan Test Acc 50.00  Test Loss nan
********************
Enter epoch epoch, learning rate = 0.09984584897756577
********************
Epoch 6 - Train Acc: 50.00 -- Train Loss nan Test Acc 50.00  Te

KeyboardInterrupt: 

In [ ]:
# save weights
save_location = "weights2.pkl"
weights = model.get_weights()

with open(save_location, 'wb') as f:
    import pickle
    pickle.dump(weights, f)